# Feature Engineering: Electric Vehicle Sales by Makers Analysis

This notebook focuses on creating meaningful features from our EV sales by manufacturers dataset to enhance our analytical capabilities. Building on the cleaned data from our previous data cleaning notebook, we'll implement several feature engineering strategies to better understand:

1. Manufacturer performance metrics
2. Temporal patterns in manufacturer performance
3. Market competition dynamics
4. Brand growth indicators
5. Segment-specific features
6. Regional performance estimation

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

In [3]:
# Suppress warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Load the cleaned datasets created in the data cleaning notebook
timestamp = '20250806'  # Using the timestamp from the files

# Main cleaned maker dataset
df_maker = pd.read_csv(f'../../data/processed/ev_sales_by_makers_cleaned_{timestamp}.csv')

# Maker monthly share dataset
df_maker_share = pd.read_csv(f'../../data/processed/ev_maker_monthly_share_{timestamp}.csv')

# Estimated maker state sales dataset
df_estimated_maker_state = pd.read_csv(f'../../data/processed/ev_estimated_maker_state_sales_{timestamp}.csv')

# Enhanced state dataset
df_state = pd.read_csv(f'../../data/processed/ev_sales_by_state_enhanced_{timestamp}.csv')

# Convert date columns to datetime
df_maker['date'] = pd.to_datetime(df_maker['date'])
df_maker_share['date'] = pd.to_datetime(df_maker_share['date'])
df_estimated_maker_state['date'] = pd.to_datetime(df_estimated_maker_state['date'])
df_state['date'] = pd.to_datetime(df_state['date'])

# Display basic information about the datasets
print("\n=== Dataset Overview ===")
print("-" * 50)
print(f"Time Range: {df_maker['date'].min().strftime('%B %Y')} to {df_maker['date'].max().strftime('%B %Y')}")
print(f"Number of Manufacturers: {df_maker['maker'].nunique()}")
print(f"Vehicle Categories: {', '.join(df_maker['vehicle_category'].unique())}")
print(f"Total Records: {len(df_maker)}")


=== Dataset Overview ===
--------------------------------------------------
Time Range: April 2021 to March 2024
Number of Manufacturers: 26
Vehicle Categories: 2-Wheelers, 4-Wheelers
Total Records: 816


## 1. Manufacturer Performance Metrics

Let's create features that help us better understand manufacturer performance in the EV market:
1. Relative market position
2. Market leadership indicators
3. Sales distribution metrics
4. Performance benchmarking

In [5]:
# Working with the main maker dataset
# Create a copy to avoid modifying the original
df_performance = df_maker.copy()

# Calculate manufacturer rank within each vehicle category and time period
df_performance["maker_rank"] = df_performance.groupby(["date", "vehicle_category"])[
    "electric_vehicles_sold"
].rank(ascending=False, method="dense")

# Identify market leaders (rank = 1) for each vehicle category and time period
df_performance["is_market_leader"] = (df_performance["maker_rank"] == 1).astype(int)

# Calculate the distance from market leader (as a percentage of leader's sales)
leader_sales = df_performance.groupby(["date", "vehicle_category"])[
    "electric_vehicles_sold"
].transform(max)
df_performance["leader_gap_pct"] = (
    (leader_sales - df_performance["electric_vehicles_sold"]) / leader_sales * 100
).round(2)
df_performance["leader_gap_pct"] = (
    df_performance["leader_gap_pct"].replace([np.inf, -np.inf], 100).fillna(100)
)

# Calculate performance compared to category average
avg_sales = df_performance.groupby(["date", "vehicle_category"])[
    "electric_vehicles_sold"
].transform("mean")
df_performance["performance_vs_avg"] = (
    (df_performance["electric_vehicles_sold"] - avg_sales) / avg_sales
).round(2)
df_performance["performance_vs_avg"] = (
    df_performance["performance_vs_avg"].replace([np.inf, -np.inf], 0).fillna(0)
)

# Create market position categories
conditions = [
    (df_performance["maker_rank"] == 1),
    (df_performance["maker_rank"] <= 3),
    (df_performance["maker_rank"] <= 5),
    (df_performance["maker_rank"] <= 10),
]
choices = ["Market Leader", "Top 3", "Top 5", "Top 10"]
df_performance["market_position"] = np.select(conditions, choices, default="Other")

# Display the new features
print("\nManufacturer Performance Metrics:")
print("-" * 50)
display(
    df_performance[
        [
            "date",
            "maker",
            "vehicle_category",
            "electric_vehicles_sold",
            "maker_rank",
            "is_market_leader",
            "leader_gap_pct",
            "performance_vs_avg",
            "market_position",
        ]
    ].head(10)
)

# Show the top manufacturers by category
print("\nTop Manufacturers by Vehicle Category (Overall):")
print("-" * 50)
top_makers = (
    df_performance.groupby(["vehicle_category", "maker"])["electric_vehicles_sold"]
    .sum()
    .reset_index()
)
top_makers = top_makers.sort_values(
    ["vehicle_category", "electric_vehicles_sold"], ascending=[True, False]
)

for category in top_makers["vehicle_category"].unique():
    print(f"\n{category} Top 5 Manufacturers:")
    display(top_makers[top_makers["vehicle_category"] == category].head(5))


Manufacturer Performance Metrics:
--------------------------------------------------


,date,maker,vehicle_category,electric_vehicles_sold,maker_rank,is_market_leader,leader_gap_pct,performance_vs_avg,market_position
0,2021-04-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
1,2022-04-01,OKAYA EV,2-Wheelers,0,13.0,0,100.0,-1.0,Other
2,2021-05-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
3,2021-06-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
4,2021-07-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
5,2021-08-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
6,2021-09-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
7,2021-10-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
8,2021-11-01,OLA ELECTRIC,2-Wheelers,0,12.0,0,100.0,-1.0,Other
9,2021-04-01,BYD India,4-Wheelers,0,6.0,0,100.0,-1.0,Top 10



Top Manufacturers by Vehicle Category (Overall):
--------------------------------------------------

2-Wheelers Top 5 Manufacturers:


,vehicle_category,maker,electric_vehicles_sold
11,2-Wheelers,OLA ELECTRIC,489473
15,2-Wheelers,TVS,272575
1,2-Wheelers,ATHER,204449
6,2-Wheelers,HERO ELECTRIC,170394
0,2-Wheelers,AMPERE,167274



4-Wheelers Top 5 Manufacturers:


,vehicle_category,maker,electric_vehicles_sold
24,4-Wheelers,Tata Motors,88935
21,4-Wheelers,Mahindra and Mahindra,41193
20,4-Wheelers,MG Motor,13753
17,4-Wheelers,BYD India,2419
18,4-Wheelers,Hyundai Motor,2076


## 2. Temporal Growth Features

Now let's create features to analyze growth patterns and temporal dynamics for each manufacturer:
1. Growth rates and momentum indicators
2. Seasonality metrics
3. Cumulative and relative growth
4. Time-based ranking shifts

In [ ]:
# Create a copy of the DataFrame
df_growth = df_maker.copy()

# Sort the data for time series calculations
df_growth = df_growth.sort_values(['maker', 'vehicle_category', 'date'])

# Add month and quarter info
df_growth['year'] = df_growth['date'].dt.year
df_growth['month'] = df_growth['date'].dt.month
df_growth['quarter'] = df_growth['date'].dt.quarter
df_growth['month_name'] = df_growth['date'].dt.strftime('%B')

# Calculate months since first appearance for each maker
first_appearance = df_growth.groupby('maker')['date'].transform('min')
df_growth['months_in_market'] = ((df_growth['date'] - first_appearance) / np.timedelta64(1, 'M')).astype(int) + 1

# Calculate growth rates
df_growth['monthly_growth_rate'] = df_growth.groupby(['maker', 'vehicle_category'])['electric_vehicles_sold'].pct_change().fillna(0)
df_growth['monthly_growth_rate'] = df_growth['monthly_growth_rate'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Calculate rolling statistics for each maker (3-month windows)
df_growth['rolling_3m_avg'] = df_growth.groupby(['maker', 'vehicle_category'])['electric_vehicles_sold'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df_growth['rolling_3m_std'] = df_growth.groupby(['maker', 'vehicle_category'])['electric_vehicles_sold'].transform(lambda x: x.rolling(window=3, min_periods=1).std())
df_growth['rolling_3m_growth'] = df_growth.groupby(['maker', 'vehicle_category'])['monthly_growth_rate'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Calculate momentum indicators
df_growth['sales_momentum'] = np.where(
    df_growth['rolling_3m_avg'] > df_growth.groupby(['maker', 'vehicle_category'])['electric_vehicles_sold'].transform(lambda x: x.shift(3).rolling(window=3, min_periods=1).mean()),
    'Positive',
    'Negative'
)

# Calculate year-over-year growth (12 month difference)
df_growth['yoy_sales'] = df_growth.groupby(['maker', 'vehicle_category', 'month'])['electric_vehicles_sold'].transform(lambda x: x / x.shift(1) - 1)
df_growth['yoy_sales'] = df_growth['yoy_sales'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Calculate rank changes over time (month to month)
df_growth['prev_month_rank'] = df_growth.groupby(['maker', 'vehicle_category'])['maker_rank'].shift(1).fillna(df_growth['maker_rank'])
df_growth['rank_change'] = df_growth['prev_month_rank'] - df_growth['maker_rank']
df_growth['rank_movement'] = np.select(
    [df_growth['rank_change'] > 0, df_growth['rank_change'] < 0, df_growth['rank_change'] == 0],
    ['Improved', 'Declined', 'Stable'],
    default='New'
)

# Display the new features
print("\nManufacturer Growth Metrics:")
print("-" * 50)
display(df_growth[['date', 'maker', 'vehicle_category', 'electric_vehicles_sold', 
                   'months_in_market', 'monthly_growth_rate', 'rolling_3m_avg',
                   'sales_momentum', 'yoy_sales', 'rank_movement']].head(15))

## 3. Market Share and Competition Features

Let's analyze the competitive landscape with features that track market share dynamics and competitive positioning:
1. Market share evolution
2. Category dominance metrics
3. Competition intensity indicators
4. Market concentration metrics

In [ ]:
# Working with the market share dataset
df_competition = df_maker_share.copy()

# Calculate the Herfindahl-Hirschman Index (HHI) for market concentration
# HHI is the sum of squared market shares (expressed as fractions)
df_competition['market_share_fraction'] = df_competition['maker_share_percent']  # Already calculated as fraction
df_competition['squared_market_share'] = df_competition['market_share_fraction'] ** 2

# Calculate HHI for each time period and vehicle category
hhi_df = df_competition.groupby(['date', 'vehicle_category'])['squared_market_share'].sum().reset_index()
hhi_df.rename(columns={'squared_market_share': 'hhi_index'}, inplace=True)
df_competition = pd.merge(df_competition, hhi_df, on=['date', 'vehicle_category'])

# Classify market concentration based on HHI
# < 0.15: Unconcentrated, 0.15-0.25: Moderately Concentrated, > 0.25: Highly Concentrated
df_competition['market_concentration'] = pd.cut(
    df_competition['hhi_index'], 
    bins=[-float('inf'), 0.15, 0.25, float('inf')],
    labels=['Unconcentrated', 'Moderately Concentrated', 'Highly Concentrated']
)

# Calculate number of effective competitors (1/HHI)
df_competition['effective_competitors'] = (1 / df_competition['hhi_index']).round(1)

# Calculate market share tiers
df_competition['market_share_tier'] = pd.cut(
    df_competition['market_share_fraction'],
    bins=[-float('inf'), 0.05, 0.1, 0.2, 0.3, float('inf')],
    labels=['<5%', '5-10%', '10-20%', '20-30%', '>30%']
)

# Calculate market share changes over time
df_competition_sorted = df_competition.sort_values(['vehicle_category', 'maker', 'date'])
df_competition_sorted['prev_market_share'] = df_competition_sorted.groupby(['vehicle_category', 'maker'])['market_share_fraction'].shift(1)
df_competition_sorted['market_share_change'] = df_competition_sorted['market_share_fraction'] - df_competition_sorted['prev_market_share']
df_competition_sorted['market_share_change_pct'] = (df_competition_sorted['market_share_change'] / df_competition_sorted['prev_market_share'] * 100).fillna(0)
df_competition_sorted['market_share_change_pct'] = df_competition_sorted['market_share_change_pct'].replace([np.inf, -np.inf], 0)

# Calculate share change status
df_competition_sorted['share_change_status'] = np.select(
    [
        df_competition_sorted['market_share_change'] > 0.02,
        df_competition_sorted['market_share_change'] > 0,
        df_competition_sorted['market_share_change'] < -0.02,
        df_competition_sorted['market_share_change'] < 0
    ],
    [
        'Strong Gain',
        'Slight Gain',
        'Strong Loss',
        'Slight Loss'
    ],
    default='Stable'
)

# Display the new features
print("\nMarket Competition Metrics:")
print("-" * 50)
display(df_competition_sorted[['date', 'vehicle_category', 'maker', 'electric_vehicles_sold',
                               'market_share_fraction', 'hhi_index', 'market_concentration',
                               'effective_competitors', 'market_share_tier',
                               'market_share_change', 'share_change_status']].head(15))

## 4. Vehicle Category Analysis Features

Let's create features that help compare and contrast the 2-wheeler and 4-wheeler segments:
1. Segment specialization metrics
2. Dual-segment presence indicators
3. Segment performance comparisons
4. Segment-specific growth rates

In [ ]:
# Identify manufacturers present in both 2W and 4W segments
makers_per_category = df_maker.groupby(['maker', 'vehicle_category']).size().unstack(fill_value=0)
makers_per_category = makers_per_category.reset_index()

if '2-Wheelers' in makers_per_category.columns and '4-Wheelers' in makers_per_category.columns:
    makers_per_category['dual_segment'] = np.where(
        (makers_per_category['2-Wheelers'] > 0) & (makers_per_category['4-Wheelers'] > 0),
        1, 0
    )
else:
    # Handle cases where one of the categories might be missing
    present_categories = [col for col in makers_per_category.columns if col != 'maker']
    makers_per_category['dual_segment'] = 0  # Default to 0
    if len(present_categories) > 1:
        # Check if any maker has presence in more than one category
        makers_per_category['dual_segment'] = np.where(
            makers_per_category[present_categories].sum(axis=1) > 1,
            1, 0
        )

# Create a dictionary mapping maker to dual_segment status
dual_segment_dict = dict(zip(makers_per_category['maker'], makers_per_category['dual_segment']))

# Add this back to the main dataframe
df_maker['dual_segment_presence'] = df_maker['maker'].map(dual_segment_dict)

# Calculate the primary segment for each manufacturer
maker_segment_sales = df_maker.groupby(['maker', 'vehicle_category'])['electric_vehicles_sold'].sum().reset_index()
maker_total_sales = maker_segment_sales.groupby('maker')['electric_vehicles_sold'].sum().reset_index()
maker_total_sales.rename(columns={'electric_vehicles_sold': 'total_sales'}, inplace=True)

# Merge to get total sales
maker_segment_sales = pd.merge(maker_segment_sales, maker_total_sales, on='maker')

# Calculate segment contribution
maker_segment_sales['segment_contribution'] = (maker_segment_sales['electric_vehicles_sold'] / maker_segment_sales['total_sales']).round(2)

# Find primary segment (highest contribution)
primary_segment = maker_segment_sales.loc[maker_segment_sales.groupby('maker')['segment_contribution'].idxmax()]
primary_segment = primary_segment[['maker', 'vehicle_category', 'segment_contribution']]
primary_segment.columns = ['maker', 'primary_segment', 'primary_segment_contribution']

# Merge these features back into the original dataframe
df_segment = df_maker.copy()
df_segment = pd.merge(df_segment, primary_segment, on='maker')

# Calculate if currently operating in primary segment
df_segment['in_primary_segment'] = (df_segment['vehicle_category'] == df_segment['primary_segment']).astype(int)

# Calculate segment specialization score
# 1.0 = fully specialized in one segment, 0.5 = equal sales in both segments
df_segment['segment_specialization'] = np.where(
    df_segment['dual_segment_presence'] == 1,
    df_segment['primary_segment_contribution'],
    1.0
)

# Display the new features
print("\nVehicle Category Analysis Metrics:")
print("-" * 50)
display(df_segment[['maker', 'vehicle_category', 'electric_vehicles_sold', 'dual_segment_presence', 
                    'primary_segment', 'primary_segment_contribution', 'in_primary_segment',
                    'segment_specialization']].drop_duplicates(subset=['maker']).head(15))

# Calculate segment-specific metrics
segment_metrics = df_maker.groupby(['date', 'vehicle_category']).agg(
    total_sales=('electric_vehicles_sold', 'sum'),
    avg_sales=('electric_vehicles_sold', 'mean'),
    max_sales=('electric_vehicles_sold', 'max'),
    min_sales=('electric_vehicles_sold', 'min'),
    makers_count=('maker', 'nunique')
).reset_index()

# Calculate segment growth rates
segment_metrics_sorted = segment_metrics.sort_values(['vehicle_category', 'date'])
segment_metrics_sorted['prev_total_sales'] = segment_metrics_sorted.groupby('vehicle_category')['total_sales'].shift(1)
segment_metrics_sorted['segment_growth'] = (
    segment_metrics_sorted['total_sales'] / segment_metrics_sorted['prev_total_sales'] - 1
).fillna(0).replace([np.inf, -np.inf], 0)

# Compare segments
print("\nSegment Comparison Metrics:")
print("-" * 50)
display(segment_metrics_sorted.head(15))

## 5. Regional Market Estimation Features

Using the estimated state-wise sales dataset, let's create features to understand regional market performance:
1. Geographic concentration metrics
2. State-wise market position indicators
3. Regional preference patterns
4. Geographic expansion metrics

In [ ]:
# Working with the estimated maker state sales
df_regional = df_estimated_maker_state.copy()

# Calculate total estimated sales per maker
maker_total_est_sales = df_regional.groupby(['date', 'maker', 'vehicle_category'])['estimated_ev_sales'].sum().reset_index()
maker_total_est_sales.rename(columns={'estimated_ev_sales': 'total_est_sales'}, inplace=True)

# Merge back to get total estimated sales
df_regional = pd.merge(df_regional, maker_total_est_sales, on=['date', 'maker', 'vehicle_category'])

# Calculate state contribution for each manufacturer (what % of sales comes from each state)
df_regional['state_contribution'] = (df_regional['estimated_ev_sales'] / df_regional['total_est_sales']).fillna(0).round(4) * 100

# Count the number of states where each maker has sales
state_count = df_regional[df_regional['estimated_ev_sales'] > 0].groupby(['date', 'maker', 'vehicle_category'])['state'].nunique().reset_index()
state_count.rename(columns={'state': 'active_states'}, inplace=True)

# Calculate geographic concentration (Herfindahl-Hirschman Index for state distribution)
df_regional['state_share_squared'] = (df_regional['state_contribution'] / 100) ** 2
geo_hhi = df_regional.groupby(['date', 'maker', 'vehicle_category'])['state_share_squared'].sum().reset_index()
geo_hhi.rename(columns={'state_share_squared': 'geo_hhi'}, inplace=True)

# Calculate total states
total_states = df_regional['state'].nunique()
print(f"\nTotal number of states in dataset: {total_states}")

# Merge state count and geographic concentration back
df_regional = pd.merge(df_regional, state_count, on=['date', 'maker', 'vehicle_category'], how='left')
df_regional = pd.merge(df_regional, geo_hhi, on=['date', 'maker', 'vehicle_category'], how='left')
df_regional['active_states'] = df_regional['active_states'].fillna(0)

# Calculate geographic reach (% of states covered)
df_regional['geographic_reach'] = (df_regional['active_states'] / total_states * 100).round(1)

# Calculate geographic concentration category
df_regional['geo_concentration'] = pd.cut(
    df_regional['geo_hhi'], 
    bins=[-float('inf'), 0.10, 0.20, 0.30, float('inf')],
    labels=['Very Diversified', 'Moderately Diversified', 'Concentrated', 'Highly Concentrated']
)

# For each maker, calculate their top state by sales
top_states = df_regional[df_regional['estimated_ev_sales'] > 0].sort_values(['date', 'maker', 'vehicle_category', 'estimated_ev_sales'], ascending=[True, True, True, False])
top_states = top_states.groupby(['date', 'maker', 'vehicle_category']).first().reset_index()
top_states = top_states[['date', 'maker', 'vehicle_category', 'state']]
top_states.rename(columns={'state': 'top_state'}, inplace=True)

# Merge back to get top state
df_regional_final = pd.merge(df_regional, top_states, on=['date', 'maker', 'vehicle_category'], how='left')
df_regional_final['is_top_state'] = (df_regional_final['state'] == df_regional_final['top_state']).astype(int)

# Display the new features
print("\nRegional Market Estimation Features:")
print("-" * 50)
display(df_regional_final[['date', 'maker', 'vehicle_category', 'state', 'estimated_ev_sales',
                          'state_contribution', 'active_states', 'geographic_reach',
                          'geo_hhi', 'geo_concentration', 'top_state', 'is_top_state']].head(15))

# Calculate regional preference (which makers dominate in which states)
state_leader = df_regional.groupby(['date', 'state', 'vehicle_category']).apply(
    lambda x: x.loc[x['estimated_ev_sales'].idxmax(), 'maker'] if len(x) > 0 and x['estimated_ev_sales'].max() > 0 else None
).reset_index()
state_leader.columns = ['date', 'state', 'vehicle_category', 'leading_maker']

print("\nState-wise Market Leaders (Sample):")
print("-" * 50)
display(state_leader.head(15))

## 6. Advanced Statistical Features

Let's create more advanced statistical features to add deeper analytical capabilities:
1. Z-scores for relative performance
2. Moving averages and volatility measures
3. Seasonal decomposition
4. Growth trend indicators

In [ ]:
# Create a copy of the main dataset
df_stats = df_maker.copy()

# Calculate z-scores for sales within each time period and category
df_stats['sales_z_score'] = df_stats.groupby(['date', 'vehicle_category'])['electric_vehicles_sold'].transform(
    lambda x: (x - x.mean()) / x.std() if x.std() != 0 else 0
)

# Calculate rolling volatility (standard deviation of growth rates)
df_stats = df_stats.sort_values(['maker', 'vehicle_category', 'date'])
df_stats['rolling_volatility'] = df_stats.groupby(['maker', 'vehicle_category'])['monthly_growth_rate'].transform(
    lambda x: x.rolling(window=3, min_periods=1).std()
)

# Calculate acceleration (change in growth rate)
df_stats['growth_acceleration'] = df_stats.groupby(['maker', 'vehicle_category'])['monthly_growth_rate'].diff()
df_stats['growth_acceleration'] = df_stats['growth_acceleration'].fillna(0)

# Calculate performance consistency 
# (Higher values indicate more consistent growth)
df_stats['consistency_score'] = 1 - df_stats['rolling_volatility'].clip(upper=1)

# Create stability indicator 
# (combines positive growth with low volatility)
df_stats['stability_indicator'] = np.where(
    (df_stats['monthly_growth_rate'] > 0) & (df_stats['rolling_volatility'] < 0.2),
    'Stable Growth',
    np.where(
        (df_stats['monthly_growth_rate'] > 0) & (df_stats['rolling_volatility'] >= 0.2),
        'Volatile Growth',
        np.where(
            (df_stats['monthly_growth_rate'] <= 0) & (df_stats['rolling_volatility'] < 0.2),
            'Stable Decline',
            'Volatile Decline'
        )
    )
)

# Calculate long-term trend (6-month moving average)
df_stats['long_term_trend'] = df_stats.groupby(['maker', 'vehicle_category'])['electric_vehicles_sold'].transform(
    lambda x: x.rolling(window=6, min_periods=1).mean()
)

# Create trend direction indicator
df_stats['trend_direction'] = np.where(
    df_stats['electric_vehicles_sold'] > df_stats['long_term_trend'],
    'Above Trend',
    'Below Trend'
)

# Display the new features
print("\nAdvanced Statistical Features:")
print("-" * 50)
display(df_stats[['date', 'maker', 'vehicle_category', 'electric_vehicles_sold',
                  'sales_z_score', 'rolling_volatility', 'growth_acceleration',
                  'consistency_score', 'stability_indicator', 'long_term_trend',
                  'trend_direction']].head(15))

## 7. Integrating All Features and Export

Now let's bring together our most valuable features into integrated datasets for further analysis:
1. Manufacturer performance dataset
2. Temporal performance dataset
3. Regional analysis dataset

In [ ]:
# Create a timestamp for versioning
timestamp = datetime.now().strftime('%Y%m%d')

# 1. Create manufacturer performance dataset
# Select key columns from the performance metrics
performance_cols = ['date', 'year', 'month', 'quarter', 'maker', 'vehicle_category', 
                   'electric_vehicles_sold', 'cumulative_sales', 'maker_rank', 
                   'market_position', 'leader_gap_pct', 'performance_vs_avg']

# Select from market share features
competition_cols = ['market_share_fraction', 'hhi_index', 'market_concentration', 
                   'effective_competitors', 'market_share_tier', 'share_change_status']

# Select from segment features
segment_cols = ['dual_segment_presence', 'primary_segment', 'primary_segment_contribution', 
               'segment_specialization']

# Create the integrated manufacturer performance dataset
df_manufacturer = df_performance[performance_cols].copy()

# Merge with competition metrics
df_competition_selected = df_competition_sorted[['date', 'maker', 'vehicle_category'] + competition_cols]
df_manufacturer = pd.merge(df_manufacturer, df_competition_selected, 
                         on=['date', 'maker', 'vehicle_category'], how='left')

# Merge with segment metrics
df_segment_selected = df_segment[['maker', 'vehicle_category'] + segment_cols].drop_duplicates()
df_manufacturer = pd.merge(df_manufacturer, df_segment_selected, 
                         on=['maker', 'vehicle_category'], how='left')

# 2. Create temporal analysis dataset
# Select key temporal columns
temporal_cols = ['date', 'maker', 'vehicle_category', 'electric_vehicles_sold', 
                'months_in_market', 'monthly_growth_rate', 'rolling_3m_avg', 
                'rolling_3m_growth', 'sales_momentum', 'rank_movement',
                'sales_z_score', 'rolling_volatility', 'stability_indicator',
                'long_term_trend', 'trend_direction']

df_temporal = df_stats[temporal_cols].copy()

# 3. Create regional analysis dataset
# Select key regional columns
regional_cols = ['date', 'maker', 'vehicle_category', 'state', 
                'estimated_ev_sales', 'state_contribution', 'state_rank',
                'active_states', 'geographic_reach', 'geo_concentration',
                'top_state', 'is_top_state']

df_regional_selected = df_regional_final[['date', 'maker', 'vehicle_category', 'state', 
                                         'estimated_ev_sales', 'state_contribution', 
                                         'active_states', 'geographic_reach', 'geo_hhi',
                                         'geo_concentration', 'top_state', 'is_top_state']].copy()

# Export the datasets
df_manufacturer.to_csv(f'../../data/processed/ev_manufacturer_performance_{timestamp}.csv', index=False)
df_temporal.to_csv(f'../../data/processed/ev_manufacturer_temporal_{timestamp}.csv', index=False)
df_regional_selected.to_csv(f'../../data/processed/ev_manufacturer_regional_{timestamp}.csv', index=False)

# Print confirmation
print("\n=== Export Complete ===")
print("-" * 50)
print(f"Files exported to data/processed/ directory with timestamp: {timestamp}")
print("\nExported Files:")
print(f"1. ev_manufacturer_performance_{timestamp}.csv")
print(f"   - Rows: {len(df_manufacturer)}")
print(f"   - Columns: {df_manufacturer.columns.tolist()}")

print(f"\n2. ev_manufacturer_temporal_{timestamp}.csv")
print(f"   - Rows: {len(df_temporal)}")
print(f"   - Columns: {df_temporal.columns.tolist()}")

print(f"\n3. ev_manufacturer_regional_{timestamp}.csv")
print(f"   - Rows: {len(df_regional_selected)}")
print(f"   - Columns: {df_regional_selected.columns.tolist()}")

## Conclusion and Next Steps

In this feature engineering notebook, we've significantly enhanced our EV manufacturer sales dataset with valuable features for deeper analysis:

### Key Features Added:
1. **Performance Metrics**: Ranking, market position, and comparison to leaders
2. **Temporal Dynamics**: Growth rates, momentum indicators, and stability metrics
3. **Competitive Landscape**: Market concentration, share metrics, and competitive positioning
4. **Segment Analysis**: Specialization, dual-segment presence, and primary segment performance
5. **Regional Insights**: Geographic reach, state contributions, and regional market leadership
6. **Advanced Statistics**: Z-scores, volatility measures, and trend indicators

### Key Insights:
- Clear differentiation between 2-wheeler and 4-wheeler manufacturer dynamics
- Identification of market leaders and their dominance patterns
- Understanding of growth trajectories and stability across manufacturers
- Geographic distribution patterns of manufacturer sales
- Competitive intensity differences across vehicle segments

### Next Steps:
1. **Exploratory Data Analysis**: Utilize these features for in-depth EV manufacturer analysis
2. **Visualizations**: Create dashboards and charts to communicate manufacturer dynamics
3. **Predictive Modeling**: Use these features to predict future market share trends
4. **Comparative Analysis**: Compare manufacturer performance across states and segments
5. **Strategic Insights**: Develop manufacturer-specific insights for market positioning

The engineered features provide a comprehensive foundation for understanding the EV manufacturer landscape in India, supporting both descriptive and predictive analytics.